# Webscraper for all Game Grumps VS Videos

### Download Modules

In [1]:
import time
import nltk
import requests
import datetime
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from nltk.corpus import stopwords
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from selenium.webdriver.support import expected_conditions as EC

nltk.download('stopwords')
nltk.download('vader_lexicon')

stopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Load in Data

In [5]:
gg_raw_data = pd.read_excel(r"ggvs_rawdata.xlsx")

gg_raw_data

,Game_Title,System,Publisher,Developer,Release_Date,Category,Result,Reference_Link
0,Hollywood Squares,NES,GameTek,Rare,1988.0,Game Show,Danny,https://www.youtube.com/watch?v=Gsno3EMZtnE&li...
1,Dragon Ball Z: Burst Limit,PlayStation 3,BNE Entertainment,Bandai Namco Holdings,2008.0,Fighting,Arin,https://www.youtube.com/watch?v=7YibTnOc9v0&li...
2,Pac-Attack,SNES,BNE Entertainment,Namco,1993.0,Puzzle,Danny,https://www.youtube.com/watch?v=8ZyJt7N_T80&li...
3,Super Tennis,SNES,Nintendo,Nintendo,1991.0,Sports,Danny,https://www.youtube.com/watch?v=bP8KauAg3vQ&li...
4,Gundam: Battle Assault 2,PlayStation,Natsume,Bandai Namco Holdings,2002.0,Fighting,Arin,https://www.youtube.com/watch?v=Pq3MKQCXsUQ&li...
...,...,...,...,...,...,...,...,...
350,Family Feud,Switch,Ubisoft,Ubisoft,2020.0,Game Show,Arin,https://www.youtube.com/watch?v=YpUeflXDoSA&t=...
351,Mario Party 2,N64,Nintendo,Hudson Soft,1999.0,Party,Arin,https://www.youtube.com/watch?v=Ld45fZtqrtI&ab...
352,WWE 2K23,Xbox One,Visual Concepts,2K Games,2023.0,Fighting,Danny,https://www.youtube.com/watch?v=oCC8JnYsOMU&ab...
353,Wheel of Fortune,PlayStation 2,Atari,Atari,2004.0,Game Show,Arin,https://www.youtube.com/watch?v=CFJ49us3yvU


In [3]:
for index, row in gg_raw_data.iterrows():
    if row['Game_Title'] == 'Tank! Tank! Tank!':
        print(index)
        break
        
gg_raw_data

223


,Game_Title,System,Publisher,Developer,Release_Date,Category,Result,Reference_Link
0,Hollywood Squares,NES,GameTek,Rare,1988.0,Game Show,Danny,https://www.youtube.com/watch?v=Gsno3EMZtnE&li...
1,Dragon Ball Z: Burst Limit,PlayStation 3,BNE Entertainment,Bandai Namco Holdings,2008.0,Fighting,Arin,https://www.youtube.com/watch?v=7YibTnOc9v0&li...
2,Pac-Attack,SNES,BNE Entertainment,Namco,1993.0,Puzzle,Danny,https://www.youtube.com/watch?v=8ZyJt7N_T80&li...
3,Super Tennis,SNES,Nintendo,Nintendo,1991.0,Sports,Danny,https://www.youtube.com/watch?v=bP8KauAg3vQ&li...
4,Gundam: Battle Assault 2,PlayStation,Natsume,Bandai Namco Holdings,2002.0,Fighting,Arin,https://www.youtube.com/watch?v=Pq3MKQCXsUQ&li...
...,...,...,...,...,...,...,...,...
349,Family Feud,Switch,Ubisoft,Ubisoft,2020.0,Game Show,Danny,https://www.youtube.com/watch?v=9GqqTqQ1VIU&t=...
350,Family Feud,Switch,Ubisoft,Ubisoft,2020.0,Game Show,Arin,https://www.youtube.com/watch?v=YpUeflXDoSA&t=...
351,Mario Party 2,N64,Nintendo,Hudson Soft,1999.0,Party,Arin,https://www.youtube.com/watch?v=Ld45fZtqrtI&ab...
352,WWE 2K23,Xbox One,Visual Concepts,2K Games,2023.0,Fighting,Danny,https://www.youtube.com/watch?v=oCC8JnYsOMU&ab...


In [9]:
gg_raw_data[353:]

,Game_Title,System,Publisher,Developer,Release_Date,Category,Result,Reference_Link
353,Wheel of Fortune,PlayStation 2,Atari,Atari,2004.0,Game Show,Arin,https://www.youtube.com/watch?v=CFJ49us3yvU


### Scrape Views, Likes, and Dates from all Selected Game Grumps Videos

In [6]:
for index, row in gg_raw_data[354:].iterrows():
    connected = False
    counter = 0
    
    while not connected:
        ##print(str(index) + '/' + str(len(gg_raw_data)))
        try:
            game_title = row['Game_Title']
            system = row['System']
            publisher = row['Publisher']
            developer = row['Developer']
            release_date = row['Release_Date']
            category = row['Category']
            result = row['Result']
            url = row['Reference_Link']
            #print(url)
            
            ### Go to Video
            driver = uc.Chrome()
            driver.get(url)
            
            ### Expand Details
            button_path = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[4]/div[1]/div/ytd-text-inline-expander/tp-yt-paper-button[1]'
            time.sleep(10)
            button = driver.find_element(by=By.XPATH, value=button_path)
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)
            
            #Scrape Views, Dates, Likes, Comments
            views = int(driver.find_element(by=By.XPATH, value='//*[@id="info"]/span[1]').text.split('views')[0].replace(',', ''))
            date = driver.find_element(by=By.XPATH, value='//*[@id="info"]/span[3]').text
            likes = driver.find_element(by=By.XPATH, value='//*[@id="segmented-like-button"]').text
            
            #Append Views, Date, Likes to DataFrame
            temp_table = pd.DataFrame()
            temp_table.at[0, 'Game_Title'] = game_title
            temp_table.at[0, 'System'] = system
            temp_table.at[0, 'Publisher'] = publisher
            temp_table.at[0, 'Developer'] = developer
            temp_table.at[0, 'Release_Date'] = release_date
            temp_table.at[0, 'Category'] = category
            temp_table.at[0, 'Result'] = result
            temp_table.at[0, 'Reference_Link'] = url
            temp_table.at[0, 'Date'] = date
            temp_table.at[0, 'Views'] = views
            temp_table.at[0, 'Likes'] = likes
            
            driver.close()
            driver.quit()
            time.sleep(10)
            
            ### Save results into excel file and delete line item - conserves RAM
            try:
                append_df1 = pd.read_excel(r'ggvs_cleaneddata.xlsx')
                save_df = pd.concat([append_df1, temp_table])
                save_df.to_excel(r'ggvs_cleaneddata.xlsx', index=False)
            except:
                temp_table.to_excel(r'ggvs_cleaneddata.xlsx', index=False)         

            connected = True
        except Exception as E:
            try:
                print(E)
                driver.close()
                driver.quit()
                time.sleep(10)
            except:
                continue
        continue

### Clean Data

In [7]:
gg_cleaned_data = pd.read_excel(r"ggvs_cleaneddata.xlsx")
print(gg_cleaned_data)
### Cleaning Functions

for index, row in gg_cleaned_data.iterrows():
    raw_likes = row['Likes']
    print(index)
    if '.' and 'K' in raw_likes:
        new_likes = str(raw_likes).replace('.', '')
        new_likes = str(new_likes).replace('K', '00')
    if 'K' in str(raw_likes) and '.' not in str(raw_likes):
        new_likes = str(raw_likes).replace('K', '000')
    
    gg_cleaned_data.at[index, 'Likes'] = new_likes
    
gg_cleaned_data.to_excel(r'ggvs_cleaneddata.xlsx', index=False) 

                     Game_Title         System          Publisher  \
0             Hollywood Squares            NES            GameTek   
1    Dragon Ball Z: Burst Limit  PlayStation 3  BNE Entertainment   
2                    Pac-Attack           SNES  BNE Entertainment   
3                  Super Tennis           SNES           Nintendo   
4      Gundam: Battle Assault 2    PlayStation            Natsume   
..                          ...            ...                ...   
350                 Family Feud         Switch            Ubisoft   
351               Mario Party 2            N64           Nintendo   
352                    WWE 2K23       Xbox One    Visual Concepts   
353            Wheel of Fortune  PlayStation 2              Atari   
354            Wheel of Fortune  PlayStation 2              Atari   

                 Developer  Release_Date   Category Result  \
0                     Rare        1988.0  Game Show  Danny   
1    Bandai Namco Holdings        2008.0   Figh

TypeError: argument of type 'int' is not iterable

In [9]:
for index, row in gg_cleaned_data[354:].iterrows():
    connected = False
    counter = 0
    
    while not connected:
        try:
            print(index)
            game_title = row['Game_Title']
            ##print(game_title)
            system = row['System']
            publisher = row['Publisher']
            developer = row['Developer']
            release_date = row['Release_Date']
            category = row['Category']
            result = row['Result']
            url = row['Reference_Link']
            date = row['Date']
            views = row['Views']
            likes = row['Likes']
            
            ### Go to Video
            driver = uc.Chrome()
            driver.get(url)
            
            ### Scroll to Enable Load All Comments
            for i in range(1, 50):
                height = 1080*i
                driver.execute_script(f"window.scrollTo(0, {height})")
                time.sleep(1.5)
                
            ### Load into Temp Table to Reduce RAM Usage
            #Append Views, Date, Likes to DataFrame
            temp_table = pd.DataFrame()
            temp_table.at[0, 'Game_Title'] = game_title
            temp_table.at[0, 'System'] = system
            temp_table.at[0, 'Publisher'] = publisher
            temp_table.at[0, 'Developer'] = developer
            temp_table.at[0, 'Release_Date'] = release_date
            temp_table.at[0, 'Category'] = category
            temp_table.at[0, 'Result'] = result
            temp_table.at[0, 'Reference_Link'] = url
            temp_table.at[0, 'Date'] = date
            temp_table.at[0, 'Views'] = views
            temp_table.at[0, 'Likes'] = likes
            
            ### Scrape All Comments
            time.sleep(5)
            comments = driver.find_elements(by=By.ID, value='content-text')
            
            if len(comments) > 0:
                cleaned_comments = []
                for comment in comments:
                    cleaned_comments.append(comment.text)

                df = pd.DataFrame(cleaned_comments, columns=['Comment'])
            
                sentiments = SentimentIntensityAnalyzer()
                df['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in df['Comment']]
                df['Negative'] = [sentiments.polarity_scores(i)['neg'] for i in df['Comment']]
                df['Neutral'] = [sentiments.polarity_scores(i)['neu'] for i in df['Comment']]
                df['Compound'] = [sentiments.polarity_scores(i)['compound'] for i in df['Comment']]
                score = df['Compound'].values
                overall_sentiment = df['Compound'].median()
                overall_average = df['Compound'].mean()

                temp_table.at[0, 'Sentiment_Score_Median'] = overall_sentiment
                temp_table.at[0, 'Sentiment_Score_Average'] = overall_average
                
            #Calculate Sentiment of Comments Mentioning Danny or Arin
                danny_alternates = ['danny', 'dan', 'daniel', 'leigh', 'avidan', 'dann', 'sexbang']
                arin_alternates = ['arin', 'aaron', 'aeron', 'egoraptor', 'egofaptor', 'erin', 'big cat']

                danny_comments = []
                arin_comments = []

                for comment in cleaned_comments:
                    if any(word.lower() in comment for word in danny_alternates):
                        danny_comments.append(comment)
                    elif any(word.lower() in comment for word in arin_alternates):
                        arin_comments.append(comment)

                df = pd.DataFrame(danny_comments, columns=['Comment'])

                sentiments = SentimentIntensityAnalyzer()
                df['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in df['Comment']]
                df['Negative'] = [sentiments.polarity_scores(i)['neg'] for i in df['Comment']]
                df['Neutral'] = [sentiments.polarity_scores(i)['neu'] for i in df['Comment']]
                df['Compound'] = [sentiments.polarity_scores(i)['compound'] for i in df['Comment']]
                score = df['Compound'].values

                overall_average = df['Compound'].mean()
                overall_sentiment = df['Compound'].median()
                temp_table.at[0, 'Danny_Sentiment_Score_average'] = overall_average
                temp_table.at[0, 'Danny_Sentiment_Score_median'] = overall_sentiment


                df = pd.DataFrame(arin_comments, columns=['Comment'])

                sentiments = SentimentIntensityAnalyzer()
                df['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in df['Comment']]
                df['Negative'] = [sentiments.polarity_scores(i)['neg'] for i in df['Comment']]
                df['Neutral'] = [sentiments.polarity_scores(i)['neu'] for i in df['Comment']]
                df['Compound'] = [sentiments.polarity_scores(i)['compound'] for i in df['Comment']]
                score = df['Compound'].values

                overall_sentiment = df['Compound'].median()
                overall_average = df['Compound'].mean()

                temp_table.at[0, 'Arin_Sentiment_Score_average'] = overall_average
                temp_table.at[0, 'Arin_Sentiment_Score_Median'] = overall_sentiment
            
            driver.close()
            driver.quit()
            time.sleep(10)
            
            ### Save results into excel file and delete line item - conserves RAM
            if np.isnan(temp_table._get_value(0, 'Sentiment_Score_Median')) == False:
                try:
                    append_df1 = pd.read_excel(r'ggvs_cleaned_sentiment_data.xlsx')
                    save_df = pd.concat([append_df1, temp_table])
                    save_df.to_excel(r'ggvs_cleaned_sentiment_data.xlsx', index=False)
                except:
                    temp_table.to_excel(r'ggvs_cleaned_sentiment_data.xlsx', index=False)         

                connected = True
        except Exception as E:
            try:
                print(E)
                driver.close()
                driver.quit()
                time.sleep(10)
            except:
                continue
        continue

354


In [32]:
for index, row in gg_raw_data.iterrows():
    connected = False
    counter = 0
    
    while not connected:
        try:
            url = row['Reference_Link']

            ### Go to Video
            driver = uc.Chrome()
            driver.get(url)

            ### Scroll to Enable Load All Comments
            for i in range(1, 150):
                height = 1080*i
                driver.execute_script(f"window.scrollTo(0, {height})")
                time.sleep(1.5)

            ### Scrape All Comments
            time.sleep(5)

            commenters_names = driver.find_elements(by=By.ID, value='author-text')

            if len(commenters_names) > 0:
                commenters = []
                for comment in commenters_names:
                    if comment.text != "Game Grumps":
                        comment_rep = comment.text
                        commenters.append(comment_rep.replace("@", ""))
                
                ### Load into Temp Table to Reduce RAM Usage
                #Append Views, Date, Likes to DataFrame
                if len(commenters) > 0:
                    temp_table = pd.DataFrame(commenters, columns=['Commenters'])
                    index_game = [str(index) + " - " + str(row['Game_Title'])]
                    index_table = pd.DataFrame(index_game, columns=['Index_Game'])
                    ### Save results into excel file and delete line item - conserves RAM
                    try:
                        append_df1 = pd.read_excel(r'ggvs_commenters.xlsx', sheet_name="Commenters")
                        save_df = pd.concat([append_df1, temp_table])
                        save_df.to_excel(r'ggvs_commenters.xlsx', sheet_name="Commenters", index=False)
                    
                        append_df1 = pd.read_excel(r'ggvs_gameindex.xlsx', sheet_name="Game_Index")
                        save_df = pd.concat([append_df1, index_table])
                        save_df.to_excel(r'ggvs_gameindex.xlsx', sheet_name="Game_Index", index=False)
                    
                    except:
                        temp_table.to_excel(r'ggvs_commenters.xlsx', sheet_name='Commenters', index=False)         
                        index_table.to_excel(r'ggvs_gameindex.xlsx', sheet_name='Game_Index', index=False)
                
                temp_table = pd.DataFrame()
                index_table = pd.DataFrame()
                connected = True
                
            driver.close()
            driver.quit()
            
        except Exception as E:
            try:
                #print(E)
                driver.close()
                driver.quit()
                time.sleep(10)
            except:
                continue
        continue

KeyboardInterrupt: 